This code is about clean all the documents and then run the scripts to generate the questions and answers based on gemini model and gpt-4o model using thread to run in parallel

In [1]:
from PyPDF2 import PdfReader, PdfWriter
import os

In [2]:
def get_pdf_text(pdf_doc):
    text = []
    pdf_reader = PdfReader(pdf_doc)
    for page_number, page in enumerate(pdf_reader.pages):
        text.append((page_number + 1, page.extract_text()))
    return text

In [3]:
def process_pd_documents(path):
    for file_index, file in enumerate(os.listdir(path)):
        if file.endswith(".pdf"):
            try:
                pages = get_pdf_text(os.path.join(path, file))
            except:
                continue

In [ ]:
process_pd_documents('documents/focus')

In [5]:
from PyPDF2 import PdfReader
import os
import pandas as pd
import re
from pathlib import Path

def get_pdf_text(pdf_path):
    try:
        pdf_reader = PdfReader(pdf_path)
        text_pages = []
        
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text.strip():
                text_pages.append(page_text)
        
        full_text = ' '.join(text_pages)
        return full_text
    
    except Exception as e:
        print(f"Error procesando {pdf_path}: {str(e)}")
        return ""

def clean_text(text):
    if not text or pd.isna(text):
        return ""

    text = str(text)

    replacements = ['\t', '\n', '\r', '#', '-', '__', '@', '/', '`', '>', '<', '{', '}', '!']
    for char in replacements:
        text = text.replace(char, ' ')
    
    text = re.sub(r'[^\w\s.,]', '', text)
    
    text = re.sub(r'\bpdf\b', '', text, flags=re.IGNORECASE)

    text = re.sub(r'\s+', ' ', text)
    
    text = text.strip()
    
    return text

def process_pdf_documents(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"El directorio {path} no existe")
    
    documents_data = []
    processed_count = 0
    error_count = 0
    
    pdf_files = [f for f in os.listdir(path) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"No se encontraron archivos PDF en {path}")
        return pd.DataFrame(columns=['filename', 'text'])
    
    print(f"Procesando {len(pdf_files)} archivos PDF...")
    
    for file in pdf_files:
        file_path = os.path.join(path, file)
        
        try:
            raw_text = get_pdf_text(file_path)
            
            if raw_text.strip():
                clean_text_content = clean_text(raw_text)
                
                if clean_text_content.strip():
                    documents_data.append({
                        'filename': file,
                        'text': clean_text_content
                    })
                    processed_count += 1
                else:
                    print(f"Advertencia: {file} no tiene contenido de texto válido después de la limpieza")
            else:
                print(f"Advertencia: {file} no contiene texto extraíble")
                
        except Exception as e:
            print(f"Error procesando {file}: {str(e)}")
            error_count += 1
            continue
    
    df = pd.DataFrame(documents_data)
    
    print(f"Procesamiento completado:")
    print(f"- Documentos procesados exitosamente: {processed_count}")
    print(f"- Errores: {error_count}")
    print(f"- Total de filas en el DataFrame: {len(df)}")
    
    return df

def get_documents_dataframe(path, save_to_csv=False, output_path=None, name= 'documentos_focus.csv'):
    df = process_pdf_documents(path)
    
    if not df.empty:
        print(f"\nDataFrame creado con {len(df)} documentos")
        print(f"Columnas: {list(df.columns)}")
        print(f"Longitud promedio del texto: {df['text'].str.len().mean():.0f} caracteres")
        
        if save_to_csv:
            if output_path is None:
                output_path = name
            df.to_csv(output_path, index=False, encoding='utf-8')
            print(f"DataFrame guardado en: {output_path}")
    else:
        print("No se pudieron procesar documentos o no se encontraron archivos válidos")
    
    return df

In [6]:
path_documentos = "documents/focus"
df_documentos_focus = get_documents_dataframe(path_documentos, save_to_csv=True)
print(df_documentos_focus.head())

Procesando 38 archivos PDF...
Advertencia: Manejo integrado plagas.pdf no contiene texto extraíble
Advertencia: la sostenibilidad de la caficultura colombiana-- Cadena Gabriel.pdf no contiene texto extraíble
Advertencia: Portada.pdf no contiene texto extraíble
Advertencia: Manejo integrado broca parte 1.pdf no contiene texto extraíble
Advertencia: arc047(04)215-230.pdf no contiene texto extraíble
Advertencia: 14 Investigaciones en roya cafeto.pdf no contiene texto extraíble
Procesamiento completado:
- Documentos procesados exitosamente: 32
- Errores: 0
- Total de filas en el DataFrame: 32

DataFrame creado con 32 documentos
Columnas: ['filename', 'text']
Longitud promedio del texto: 121792 caracteres
DataFrame guardado en: documentos_focus.csv
                      filename  \
0                  avt0349.pdf   
1      yvbravoclmcordobago.pdf   
2  GuiaMasAgronomia-nov_16.pdf   
3                   bot036.pdf   
4                     CAP4.pdf   

                                         

In [7]:
df_documentos_focus.head()

,filename,text
0,avt0349.pdf,349349 Gerencia Técnica Programa de Investigac...
1,yvbravoclmcordobago.pdf,1 Manej o Agroecológico del Cultivo de C afé C...
2,GuiaMasAgronomia-nov_16.pdf,GUÍA Los trabajos suscritos por el personal té...
3,bot036.pdf,Ministro de Hacienda y Crédito Público Ministr...
4,CAP4.pdf,El Control Natural de Insectos en el Ecosistem...


In [8]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [9]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-mini-2025-04-14")

In [41]:
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
import re
from typing import List

def generar_preguntas_para_texto(texto: str, modelo, num_preguntas: int = 10) -> List[str]:
    system_prompt = f"""Eres un ingeniero agrónomo y caficultor colombiano experto en café. 
    Tu tarea es generar exactamente {num_preguntas} preguntas técnicas específicas sobre café basadas ÚNICAMENTE en el contenido del texto proporcionado.
    Note: Ten en cuenta que será preguntas que se usarán para hacer luego un fine tuning a una LLM.
    
    CRITERIOS OBLIGATORIOS para las preguntas:
    - Deben ser preguntas generales, evitar hacer preguntas de la investigación exclusiva como: Abono utilizado en la finca El Agrado durante la investigación?
    - No generes preguntas compuestas, solo una pregunta.
    - Deben ser respondibles EXCLUSIVAMENTE con la información contenida en el texto
    - Enfoque técnico: producción, cultivo, procesamiento, variedades, enfermedades, plagas, calidad, postcosecha, beneficio, etc.
    - Usar terminología técnica precisa del sector cafetero que se maneje en el documento
    - Variar entre aspectos agronómicos, fitosanitarios, de calidad y procesamiento según el contenido
    - Ser específicas al contenido presente en el documento
    
    FORMATO REQUERIDO:
    1. [Primera pregunta técnica]
    2. [Segunda pregunta técnica]
    ...
    {num_preguntas}. [Última pregunta técnica]
    
    IMPORTANTE: Cada pregunta debe tener su respuesta claramente presente en el texto proporcionado."""
    
    messages = [
        SystemMessage(system_prompt),
        HumanMessage(f"Texto a analizar:\n\n{texto}")
    ]
    
    respuesta = modelo.invoke(messages)
    
    if hasattr(respuesta, 'content'):
        contenido = respuesta.content
    else:
        contenido = str(respuesta)
    
    preguntas = extraer_preguntas_numeradas(contenido, num_preguntas)
    
    return preguntas

def extraer_preguntas_numeradas(texto: str, num_esperado: int) -> List[str]:

    patron = r'^\s*\d+[\.\)\-]\s*(.+?)(?=^\s*\d+[\.\)\-]|\Z)'
    preguntas = re.findall(patron, texto, re.MULTILINE | re.DOTALL)
    
    preguntas_limpias = []
    for pregunta in preguntas:
        pregunta_limpia = pregunta.strip().replace('\n', ' ')
        pregunta_limpia = re.sub(r'\s+', ' ', pregunta_limpia)
        if pregunta_limpia:
            preguntas_limpias.append(pregunta_limpia)
    
    if len(preguntas_limpias) != num_esperado:
        lineas = [linea.strip() for linea in texto.split('\n') if linea.strip()]
        preguntas_limpias = []
        for linea in lineas:
            linea_limpia = re.sub(r'^\s*\d+[\.\)\-]\s*', '', linea)
            if linea_limpia and len(linea_limpia) > 10:
                preguntas_limpias.append(linea_limpia)
    
    return preguntas_limpias[:num_esperado]

def procesar_documentos_y_generar_preguntas(df_documentos: pd.DataFrame, modelo, num_preguntas: int = 10) -> pd.DataFrame:
    
    resultados = []
    
    for index, row in df_documentos.iterrows():
        filename = row['filename']
        texto = row['text']
        
        print(f"Procesando archivo: {filename} (fila {index + 1}/{len(df_documentos)})")
        
        try:
            preguntas = generar_preguntas_para_texto(texto, modelo, num_preguntas)
            for pregunta in preguntas:
                resultados.append({
                    'texto': texto,
                    'preguntas': pregunta
                })
            
            print(f"  ✓ Generadas {len(preguntas)} preguntas")
            
        except Exception as e:
            print(f"  ✗ Error procesando {filename}: {str(e)}")
            for _ in range(num_preguntas):
                resultados.append({
                    'texto': texto,
                    'preguntas': f"[Error generando pregunta: {str(e)}]"
                })
    
    df_resultado = pd.DataFrame(resultados)
    
    print(f"\nProcesamiento completado:")
    print(f"- Documentos procesados: {len(df_documentos)}")
    print(f"- Total de preguntas generadas: {len(df_resultado)}")
    print(f"- Preguntas por documento: {num_preguntas}")
    
    return df_resultado

In [48]:
df_preguntas = procesar_documentos_y_generar_preguntas(df_documentos, model, num_preguntas=10)

Procesando archivo: avt0506.pdf (fila 1/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0411.pdf (fila 2/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0536.pdf (fila 3/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0349.pdf (fila 4/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0388.pdf (fila 5/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0370.pdf (fila 6/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0398.pdf (fila 7/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0569.pdf (fila 8/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0477.pdf (fila 9/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0422.pdf (fila 10/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0492.pdf (fila 11/187)
  ✓ Generadas 10 preguntas
Procesando archivo: arc057(03)232-242.pdf (fila 12/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0358.pdf (fila 13/187)
  ✓ Generadas 10 preguntas
Procesando archivo: avt0403.pdf (fila 14/187)
  ✓ 

In [50]:
#to csv
df_preguntas.to_csv("documentos_preguntas.csv")

In [49]:
df_preguntas['texto'][0]

'Fraccionamiento de la fertilización en el cultivo del café La fertilización constituye una práctica clave para la producción de café. Su eficiencia depende, entre otros aspectos, de la oportunidad con la que se realice Arcila, 2007, de una adecuada recomendación, de las características del suelo y de las condiciones de clima. Considerando la diversidad de suelos en los que se cultiva el café en Colombia, el costo de los fertilizantes y las variaciones climáticas, los productores y la institucionalidad cafetera requieren de alternativas que conduzcan a incrementar la productividad en el cultivo del café. En la zona cafetera colombiana, durante décadas, se ha recomendado que el fertilizante requerido por año para cafetales en producción sea suministrado cada seis meses, basados en el hecho que mediante dicha práctica se obtienen los mayores beneficios desde el punto de vista económico Mestre y Uribe, 1980. Pese a esto, los resultados de las investigaciones que han dado sustento a tal re

In [33]:
df_preguntas['preguntas'][0]

'¿Qué factores determinan la eficiencia de la fertilización en el cultivo del café según el documento?'

In [35]:
df_preguntas['preguntas'][1]

'¿Cuál es la recomendación tradicional sobre la frecuencia anual de fertilización en cafetales en producción en Colombia?'

In [36]:
df_preguntas['preguntas'][2]

'¿Cómo se realizó el fraccionamiento de la fertilización en los cuatro tratamientos establecidos durante la fase reproductiva según el estudio?'

In [37]:
df_preguntas['preguntas'][3]

'¿Qué tipo de fertilizantes se utilizaron para suplir los requerimientos de nitrógeno, fósforo, potasio y magnesio en las parcelas estudiadas?'

In [38]:
df_preguntas['preguntas'][4]

'¿Cuál fue el criterio clave para seleccionar el momento de aplicación de fertilizantes en relación con la humedad del suelo?'

In [58]:
df_preguntas['preguntas'][1000]

'¿Cuáles son las dos formas principales de azufre presentes en los suelos de la zona cafetera colombiana?'

In [61]:
path_documentos = "documents/revistas"
df_documentos = get_documents_dataframe(path_documentos, save_to_csv=True, name='documentos_revistas_preguntas.csv')
print(df_documentos.head())

Procesando 200 archivos PDF...
Error procesando documents/revistas/1.Aplicacion.pdf: PyCryptodome is required for AES algorithm
Advertencia: 1.Aplicacion.pdf no contiene texto extraíble
Error procesando documents/revistas/arc061(01)055-066.pdf: PyCryptodome is required for AES algorithm
Advertencia: arc061(01)055-066.pdf no contiene texto extraíble
Error procesando documents/revistas/3.Dispositivo.pdf: PyCryptodome is required for AES algorithm
Advertencia: 3.Dispositivo.pdf no contiene texto extraíble
Error procesando documents/revistas/2.Descripcion.pdf: PyCryptodome is required for AES algorithm
Advertencia: 2.Descripcion.pdf no contiene texto extraíble
Error procesando documents/revistas/arc069(01)055-067.pdf: PyCryptodome is required for AES algorithm
Advertencia: arc069(01)055-067.pdf no contiene texto extraíble
Error procesando documents/revistas/Revista69(1)-Web1.pdf: PyCryptodome is required for AES algorithm
Advertencia: Revista69(1)-Web1.pdf no contiene texto extraíble
Error

In [67]:
df_documentos.head()

,filename,text
0,Articulo_AspectosBiologicos.pdf,"Cenicafé, 742 20237Giraldo Jaramillo, M. 2023...."
1,arc061(03)241-251.pdf,"Cenicafé, 613241 251. 2010 241EVALUACIÓN DE UN..."
2,5.Hemiptera.pdf,"Cenicafé, 67173 80. 201673HEMIPTERA COCCOIDEA ..."
3,arc070(02)091-097.pdf,"Cenicafé, 70291 97. 201991LÓPEZ F., F. LAITON ..."
4,arc069(01)032-039.pdf,"Cenicafé, 69132 39. 201832GIRALDO J., M. Efect..."


In [69]:
df_documentos['text'][0]

'Cenicafé, 742 20237Giraldo Jaramillo, M. 2023. Aspectos biológicos y reproductivos de Hypothenemus hampei en tres temperaturas constantes en Coffea arabica en laboratorio. Revista Cenicafé, 742, e74201. https doi.org 10.38141 10778 74201 Hypothenemus hampei Coleoptera Curculionidae es la mayor plaga del cultivo del café alrededor del mundo. En Colombia afecta a la caficultura ocasionando daños al fruto que genera pérdidas económicas a los caficultores cada año. El presente estudio tiene como objetivo determinar los aspectos biológicos y reproductivos de H. hampei, incluyendo una tabla de vida parcial de fertilidad en tres temperaturas constantes 20, 25 y 301C en condiciones de laboratório, sobre Coffea arabica var. Tabi. La broca del café completó su ciclo huevo adulto entre 20C y 30C. En la temperatura de 25C se presentaron los mayores valores de la tasa neta reproductiva 19,5, tasa de crecimiento intrínseca 0,08 y porcentaje de sobrevivencia de huevo adulto 87. Conocer el comportame

In [68]:
len(df_documentos['text'][0])

27268

In [70]:
df_documentos['longitud'] = df_documentos['text'].str.split().str.len()

In [72]:
df_documentos.head()

,filename,text,longitud
0,Articulo_AspectosBiologicos.pdf,"Cenicafé, 742 20237Giraldo Jaramillo, M. 2023....",4248
1,arc061(03)241-251.pdf,"Cenicafé, 613241 251. 2010 241EVALUACIÓN DE UN...",4682
2,5.Hemiptera.pdf,"Cenicafé, 67173 80. 201673HEMIPTERA COCCOIDEA ...",3800
3,arc070(02)091-097.pdf,"Cenicafé, 70291 97. 201991LÓPEZ F., F. LAITON ...",4693
4,arc069(01)032-039.pdf,"Cenicafé, 69132 39. 201832GIRALDO J., M. Efect...",3731


In [11]:
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
import re
from typing import List

def determinar_num_preguntas(longitud: int) -> int:
    """
    Determina el número de preguntas a generar basado en la longitud del texto
    """
    if longitud < 5000:
        return 8
    elif 5001 <= longitud <= 10000:
        return 10
    elif 10001 <= longitud <= 15000:
        return 15
    elif 15001 <= longitud <= 20000:
        return 15
    elif 20001 <= longitud <= 30000:
        return 20
    elif 30001 <= longitud <= 40000:
        return 25
    else:  # mayor a 40000
        return 30

def generar_preguntas_para_texto(texto: str, modelo, num_preguntas: int = 10) -> List[str]:
    system_prompt = f"""Eres un ingeniero agrónomo y caficultor colombiano experto en café. 
    Tu tarea es generar exactamente {num_preguntas} preguntas técnicas específicas sobre café basadas ÚNICAMENTE en el contenido del texto proporcionado.
    Note: Ten en cuenta que será preguntas que se usarán para hacer luego un fine tuning a una LLM.
    
    CRITERIOS OBLIGATORIOS para las preguntas:
    - Enfocate mucho en control de enfermedades, plagas, pràcticas agrìcolas, manejo de cultivo
    - Deben ser preguntas generales, evitar hacer preguntas de la investigación exclusiva como: Abono utilizado en la finca El Agrado durante la investigación?
    - No generes preguntas compuestas, solo una pregunta.
    - Deben ser respondibles EXCLUSIVAMENTE con la información contenida en el texto
    - Enfoque técnico: producción, cultivo, procesamiento, variedades, enfermedades, plagas, calidad, postcosecha, beneficio, etc.
    - Usar terminología técnica precisa del sector cafetero que se maneje en el documento
    - Variar entre aspectos agronómicos, fitosanitarios, de calidad y procesamiento según el contenido
    - Ser específicas al contenido presente en el documento
    
    FORMATO REQUERIDO:
    1. [Primera pregunta técnica]
    2. [Segunda pregunta técnica]
    ...
    {num_preguntas}. [Última pregunta técnica]
    
    IMPORTANTE: Cada pregunta debe tener su respuesta claramente presente en el texto proporcionado."""
    
    messages = [
        SystemMessage(system_prompt),
        HumanMessage(f"Texto a analizar:\n\n{texto}")
    ]
    
    respuesta = modelo.invoke(messages)
    
    if hasattr(respuesta, 'content'):
        contenido = respuesta.content
    else:
        contenido = str(respuesta)
    
    preguntas = extraer_preguntas_numeradas(contenido, num_preguntas)
    
    return preguntas

def extraer_preguntas_numeradas(texto: str, num_esperado: int) -> List[str]:

    patron = r'^\s*\d+[\.\)\-]\s*(.+?)(?=^\s*\d+[\.\)\-]|\Z)'
    preguntas = re.findall(patron, texto, re.MULTILINE | re.DOTALL)
    
    preguntas_limpias = []
    for pregunta in preguntas:
        pregunta_limpia = pregunta.strip().replace('\n', ' ')
        pregunta_limpia = re.sub(r'\s+', ' ', pregunta_limpia)
        if pregunta_limpia:
            preguntas_limpias.append(pregunta_limpia)
    
    if len(preguntas_limpias) != num_esperado:
        lineas = [linea.strip() for linea in texto.split('\n') if linea.strip()]
        preguntas_limpias = []
        for linea in lineas:
            linea_limpia = re.sub(r'^\s*\d+[\.\)\-]\s*', '', linea)
            if linea_limpia and len(linea_limpia) > 10:
                preguntas_limpias.append(linea_limpia)
    
    return preguntas_limpias[:num_esperado]

def procesar_documentos_y_generar_preguntas(df_documentos: pd.DataFrame, modelo) -> pd.DataFrame:
    if 'longitud' not in df_documentos.columns:
        raise ValueError("El DataFrame debe tener una columna 'longitud' con el número de palabras")
    
    resultados = []
    
    for index, row in df_documentos.iterrows():
        filename = row['filename']
        texto = row['text']
        longitud = row['longitud']
        
        num_preguntas = determinar_num_preguntas(longitud)
        
        print(f"Procesando archivo: {filename} (fila {index + 1}/{len(df_documentos)})")
        print(f"  Longitud: {longitud} palabras → {num_preguntas} preguntas")
        
        try:
            preguntas = generar_preguntas_para_texto(texto, modelo, num_preguntas)
            for pregunta in preguntas:
                resultados.append({
                    'texto': texto,
                    'preguntas': pregunta,
                    'longitud_original': longitud,
                    'num_preguntas_generadas': num_preguntas
                })
            
            print(f"  ✓ Generadas {len(preguntas)} preguntas")
            
        except Exception as e:
            print(f"  ✗ Error procesando {filename}: {str(e)}")
            for _ in range(num_preguntas):
                resultados.append({
                    'texto': texto,
                    'preguntas': f"[Error generando pregunta: {str(e)}]",
                    'longitud_original': longitud,
                    'num_preguntas_generadas': num_preguntas
                })
    
    df_resultado = pd.DataFrame(resultados)
    
    print(f"\nProcesamiento completado:")
    print(f"- Documentos procesados: {len(df_documentos)}")
    print(f"- Total de preguntas generadas: {len(df_resultado)}")
    
    df_stats = df_documentos.copy()
    df_stats['num_preguntas'] = df_stats['longitud'].apply(determinar_num_preguntas)
    
    print(f"\nDistribución de preguntas por longitud:")
    stats_summary = df_stats.groupby('num_preguntas').agg({
        'longitud': ['count', 'mean', 'min', 'max']
    }).round(2)
    print(stats_summary)
    
    return df_resultado

In [76]:
df_preguntas_revistas = procesar_documentos_y_generar_preguntas(df_documentos, model)

Procesando archivo: Articulo_AspectosBiologicos.pdf (fila 1/169)
  Longitud: 4248 palabras → 8 preguntas
  ✓ Generadas 8 preguntas
Procesando archivo: arc061(03)241-251.pdf (fila 2/169)
  Longitud: 4682 palabras → 8 preguntas
  ✓ Generadas 8 preguntas
Procesando archivo: 5.Hemiptera.pdf (fila 3/169)
  Longitud: 3800 palabras → 8 preguntas
  ✓ Generadas 8 preguntas
Procesando archivo: arc070(02)091-097.pdf (fila 4/169)
  Longitud: 4693 palabras → 8 preguntas
  ✓ Generadas 8 preguntas
Procesando archivo: arc069(01)032-039.pdf (fila 5/169)
  Longitud: 3731 palabras → 8 preguntas
  ✓ Generadas 8 preguntas
Procesando archivo: Rev._62(1)._art_1._Conservacion_de_suelos.pdf (fila 6/169)
  Longitud: 4416 palabras → 8 preguntas
  ✓ Generadas 8 preguntas
Procesando archivo: arc71(2)007-020.pdf (fila 7/169)
  Longitud: 5784 palabras → 10 preguntas
  ✓ Generadas 10 preguntas
Procesando archivo: arc072(02)e72202.pdf (fila 8/169)
  Longitud: 5006 palabras → 10 preguntas
  ✓ Generadas 10 preguntas
Pro

In [77]:
df_preguntas_revistas.shape

(2042, 4)

In [ ]:
path_documentos = "documents/libros"
df_documentos_libros = get_documents_dataframe(path_documentos, save_to_csv=True, name='documentos_libros_preguntas.csv')
print(df_documentos_libros.head())

Procesando 38 archivos PDF...
Advertencia: Portada.pdf no contiene texto extraíble
Advertencia: Manejo integrado broca parte 1.pdf no contiene texto extraíble
Advertencia: 32 Resistencia genética a roya cafeto.pdf no contiene texto extraíble
Advertencia: Manejo integrado enfermedades.pdf no contiene texto extraíble
Error procesando documents/libros/libroClima.pdf: PyCryptodome is required for AES algorithm
Advertencia: libroClima.pdf no contiene texto extraíble
Error procesando documents/libros/lib37751.pdf: PyCryptodome is required for AES algorithm
Advertencia: lib37751.pdf no contiene texto extraíble
Procesamiento completado:
- Documentos procesados exitosamente: 32
- Errores: 0
- Total de filas en el DataFrame: 32

DataFrame creado con 32 documentos
Columnas: ['filename', 'text']
Longitud promedio del texto: 300066 caracteres
DataFrame guardado en: documentos_libros_preguntas.csv
                      filename  \
0                  Arboles.pdf   
1                 LIBRORED.pdf   
2

In [79]:
df_documentos_libros.shape

(32, 2)

In [13]:
df_documentos_focus['longitud'] = df_documentos_focus['text'].str.split().str.len()

In [14]:
df_preguntas_libros = procesar_documentos_y_generar_preguntas(df_documentos_focus, model)

Procesando archivo: avt0349.pdf (fila 1/32)
  Longitud: 7009 palabras → 10 preguntas
  ✓ Generadas 10 preguntas
Procesando archivo: yvbravoclmcordobago.pdf (fila 2/32)
  Longitud: 23894 palabras → 20 preguntas
  ✓ Generadas 20 preguntas
Procesando archivo: GuiaMasAgronomia-nov_16.pdf (fila 3/32)
  Longitud: 32433 palabras → 25 preguntas
  ✓ Generadas 25 preguntas
Procesando archivo: bot036.pdf (fila 4/32)
  Longitud: 16038 palabras → 15 preguntas
  ✓ Generadas 15 preguntas
Procesando archivo: CAP4.pdf (fila 5/32)
  Longitud: 19047 palabras → 15 preguntas
  ✓ Generadas 15 preguntas
Procesando archivo: Bustillo-2007-Biocontrol-of-coffee-berry-borer.pdf (fila 6/32)
  Longitud: 3816 palabras → 8 preguntas
  ✓ Generadas 8 preguntas
Procesando archivo: avt0478.pdf (fila 7/32)
  Longitud: 3721 palabras → 8 preguntas
  ✓ Generadas 8 preguntas
Procesando archivo: Agentic architecture.docx.pdf (fila 8/32)
  Longitud: 7777 palabras → 10 preguntas
  ✓ Generadas 10 preguntas
Procesando archivo: bro

In [84]:
df_preguntas.head()

,texto,preguntas
0,Fraccionamiento de la fertilización en el cult...,¿Cuál fue la variedad de café utilizada en la ...
1,Fraccionamiento de la fertilización en el cult...,¿Qué tipos de fertilizantes se emplearon para ...
2,Fraccionamiento de la fertilización en el cult...,¿Cuáles fueron las texturas predominantes de l...
3,Fraccionamiento de la fertilización en el cult...,¿En qué momento se programó la fertilización p...
4,Fraccionamiento de la fertilización en el cult...,¿Cuáles fueron las frecuencias de aplicación d...


In [83]:
df_preguntas_libros.head()

,texto,preguntas,longitud_original,num_preguntas_generadas
0,1 2 Créditos Edición de textos Sandr a Milena ...,¿Qué objetivo tienen los sistemas agroforestal...,21503,20
1,1 2 Créditos Edición de textos Sandr a Milena ...,¿Cuáles son las principales características ag...,21503,20
2,1 2 Créditos Edición de textos Sandr a Milena ...,"Según el documento, ¿qué porcentaje del área c...",21503,20
3,1 2 Créditos Edición de textos Sandr a Milena ...,¿En el sistema tradicional de explotación del ...,21503,20
4,1 2 Créditos Edición de textos Sandr a Milena ...,¿Cuál es la densidad de siembra promedio de ca...,21503,20


In [15]:
df_preguntas_libros.drop(['longitud_original','num_preguntas_generadas'],axis=1, inplace=True)

In [87]:
df_preguntas_libros.drop(['longitud_original','num_preguntas_generadas'],axis=1, inplace=True)
df_preguntas_revistas.drop(['longitud_original','num_preguntas_generadas'],axis=1, inplace=True)

In [88]:
df_preguntas_completo = pd.concat([df_preguntas_libros, df_preguntas_revistas, df_preguntas], 
                       ignore_index=True)

In [90]:
df_preguntas_completo.shape

(4635, 2)

In [91]:
df_preguntas_completo.to_csv("preguntas.csv")

In [95]:
df_preguntas_completo.head()

,texto,preguntas
0,1 2 Créditos Edición de textos Sandr a Milena ...,¿Qué objetivo tienen los sistemas agroforestal...
1,1 2 Créditos Edición de textos Sandr a Milena ...,¿Cuáles son las principales características ag...
2,1 2 Créditos Edición de textos Sandr a Milena ...,"Según el documento, ¿qué porcentaje del área c..."
3,1 2 Créditos Edición de textos Sandr a Milena ...,¿En el sistema tradicional de explotación del ...
4,1 2 Créditos Edición de textos Sandr a Milena ...,¿Cuál es la densidad de siembra promedio de ca...


In [16]:
def responder_preguntas(df_preguntas, modelo):
    
    respuestas = []
    
    for index, row in df_preguntas.iterrows():
        texto = row['texto']
        pregunta = row['preguntas']
        
        print(f"Procesando pregunta {index + 1}/{len(df_preguntas)}")
        
        system_prompt = """Eres un ingeniero agrónomo experto en café de Colombia. 
        Responde la pregunta de forma directa y precisa usando únicamente la información del texto.
        
        INSTRUCCIONES:
        - Da una respuesta directa sin mencionar "según el texto" o "el texto dice"
        - No incluyas referencias textuales ni citas
        - Respuesta concisa y técnica"""
        
        human_message = f"""Texto: {texto}

        Pregunta: {pregunta}

        Respuesta:"""
        
        messages = [
            SystemMessage(system_prompt),
            HumanMessage(human_message)
        ]
        
        respuesta = modelo.invoke(messages)
        
        if hasattr(respuesta, 'content'):
            contenido = respuesta.content
        else:
            contenido = str(respuesta)
        
        respuestas.append(contenido)

    df_preguntas['respuestas'] = respuestas
    
    return df_preguntas

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model2 = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=GOOGLE_API_KEY,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [20]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-mini-2025-04-14",
    max_completion_tokens=800)

In [ ]:
<#This script processes a DataFrame of texts and questions, using two language models to generate answers. It handles rate limits and saves progress incrementally.#>

import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import time
import random

def procesar_pregunta(args):
    index, texto, pregunta, modelo, modelo2, total = args
    print(f"Procesando pregunta {index + 1}/{total}")
    
    system_prompt = """Eres un ingeniero agrónomo experto en café de Colombia. 
    Responde la pregunta de forma directa y precisa usando únicamente la información del texto.
    
    INSTRUCCIONES:
    - Dar respuestas técnicas y precisas
    - Da una respuesta directa sin mencionar "según el texto" o "el texto dice"
    - No incluyas referencias textuales ni citas
    - Respuesta concisa y técnica
    - Solo la información esencial"""
    
    human_message = f"""Texto: {texto}

Pregunta: {pregunta}

Respuesta:"""
    
    messages = [SystemMessage(system_prompt), HumanMessage(human_message)]
    
    for intento in range(3):
        try:
            if (index + 1) % 20 == 0:
                print(f"Pausa de 1 minuto después de {index + 1} preguntas...")
                time.sleep(60)
            
            respuesta = modelo.invoke(messages)
            if hasattr(respuesta, 'content'):
                contenido = respuesta.content
            else:
                contenido = str(respuesta)
            return index, contenido
            
        except Exception as e:
            if "rate_limit" in str(e).lower() or "429" in str(e):
                wait_time = (2 ** intento) + random.uniform(1, 3)
                print(f"Rate limit en pregunta {index + 1}, esperando {wait_time:.1f}s")
                time.sleep(wait_time)
            else:
                print(f"Error con modelo principal en pregunta {index + 1}, intentando fallback...")
                break

    for intento in range(3):
        try:
            respuesta = modelo2.invoke(messages)
            contenido = respuesta.content
            print(f"✓ Fallback Claude exitoso en pregunta {index + 1}")
            return index, contenido
            
        except Exception as e:
            if "rate_limit" in str(e).lower() or "429" in str(e):
                wait_time = (2 ** intento) + random.uniform(1, 3)
                print(f"Rate limit en Claude pregunta {index + 1}, esperando {wait_time:.1f}s")
                time.sleep(wait_time)
            else:
                print(f"Error con Claude en pregunta {index + 1}: {str(e)}")
                import traceback
                traceback.print_exc()
    
    return index, "[Error: Ambos modelos fallaron]"

def responder_preguntas(df_preguntas, modelo, modelo2):
    os.makedirs("questionsfocus", exist_ok=True)
    
    total = len(df_preguntas)
    args = [(i, row['texto'], row['preguntas'], modelo, modelo2, total) 
            for i, (_, row) in enumerate(df_preguntas.iterrows())]
    
    respuestas = [''] * total
    completadas = 0
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(procesar_pregunta, arg): arg[0] for arg in args}
        
        for future in as_completed(futures):
            index, respuesta = future.result()
            respuestas[index] = respuesta
            completadas += 1
            
            if completadas % 100 == 0:
                df_temp = df_preguntas.copy()
                df_temp['respuestas'] = respuestas
                df_temp_completed = df_temp[df_temp['respuestas'] != '']
                batch_num = completadas // 100
                filename = f"questionsfocus/batch_{batch_num:03d}_parcial.csv"
                df_temp_completed.to_csv(filename, index=False)
                print(f"💾 Guardadas {completadas} respuestas en {filename}")
    
    df_resultado = df_preguntas.copy()
    df_resultado['respuestas'] = respuestas
    df_resultado.to_csv("questionsfocus/resultado_final.csv", index=False)
    print(f"💾 Guardado resultado final con {total} preguntas")
    
    return df_resultado

In [23]:
df_con_respuestas = responder_preguntas(df_preguntas_libros, model, model2)

Procesando pregunta 1/456
Procesando pregunta 2/456
Procesando pregunta 3/456
Procesando pregunta 4/456
Procesando pregunta 5/456
Procesando pregunta 6/456
Procesando pregunta 7/456
Procesando pregunta 8/456
Procesando pregunta 9/456
Procesando pregunta 10/456
Procesando pregunta 11/456
Procesando pregunta 12/456
Procesando pregunta 13/456
Procesando pregunta 14/456
Procesando pregunta 15/456
Procesando pregunta 16/456
Procesando pregunta 17/456
Procesando pregunta 18/456
Procesando pregunta 19/456
Procesando pregunta 20/456
Pausa de 1 minuto después de 20 preguntas...
Procesando pregunta 21/456
Procesando pregunta 22/456
Procesando pregunta 23/456
Procesando pregunta 24/456
Procesando pregunta 25/456
Procesando pregunta 26/456
Procesando pregunta 27/456
Procesando pregunta 28/456
Procesando pregunta 29/456
Procesando pregunta 30/456
Procesando pregunta 31/456
Procesando pregunta 32/456
Procesando pregunta 33/456
Procesando pregunta 34/456
Procesando pregunta 35/456
Procesando pregunta

In [24]:
df_preguntas_total = pd.read_csv("questions/resultado_final.csv")

In [25]:
df_preguntas_completo = pd.concat([df_preguntas_total,df_con_respuestas], 
                       ignore_index=True)

In [26]:
df_preguntas_completo.shape

(5091, 3)

In [27]:
df_preguntas_completo.to_csv("respuestas_finaltotal.csv")

In [174]:
df_con_respuestas.head()

,texto,preguntas,respuestas
0,1 2 Créditos Edición de textos Sandr a Milena ...,¿Qué objetivo tienen los sistemas agroforestal...,Los sistemas agroforestales cafeteros tienen c...
1,1 2 Créditos Edición de textos Sandr a Milena ...,¿Cuáles son las principales características ag...,Las principales características agronómicas de...
2,1 2 Créditos Edición de textos Sandr a Milena ...,"Según el documento, ¿qué porcentaje del área c...","El 33,1% del área cafetera en Colombia se encu..."
3,1 2 Créditos Edición de textos Sandr a Milena ...,¿En el sistema tradicional de explotación del ...,Las variedades principales empleadas bajo somb...
4,1 2 Créditos Edición de textos Sandr a Milena ...,¿Cuál es la densidad de siembra promedio de ca...,La densidad de siembra promedio de cafetos por...


In [ ]:
def test_preguntas_cafe_sin_texto():
    NUM_PREGUNTAS = 10
    
    system_prompt = f"""Eres un ingeniero agrónomo y caficultor colombiano experto en café. 
Tu tarea es generar exactamente {NUM_PREGUNTAS} preguntas técnicas importantes sobre café que un especialista en café debería conocer.

CRITERIOS para las preguntas:
- Deben ser preguntas técnicas específicas del sector cafetero
- Enfoque en aspectos agronómicos, fitosanitarios, de calidad y procesamiento
- Usar terminología técnica precisa del sector cafetero
- Cubrir diferentes áreas: producción, cultivo, procesamiento, variedades, enfermedades, plagas, calidad, postcosecha, beneficio
- Evitar preguntas muy genéricas o básicas
- Preguntas que un caficultor profesional encuentre relevantes y útiles

ÁREAS TÉCNICAS A INCLUIR:
- Aspectos agronómicos: densidades de siembra, manejo de suelos, nutrición, podas
- Fitosanidad: enfermedades, plagas, síntomas, tratamientos, resistencia
- Variedades: características varietales, resistencias, adaptaciones climáticas
- Procesamiento: métodos de beneficio, fermentación, secado, despulpado
- Calidad: atributos sensoriales, defectos, parámetros físicos, taza
- Postcosecha: almacenamiento, trilla, clasificación

FORMATO REQUERIDO:
1. [Primera pregunta técnica]
2. [Segunda pregunta técnica]
...
{NUM_PREGUNTAS}. [Última pregunta técnica]"""

    human_message = "Genera las preguntas técnicas de café más importantes que un especialista debería conocer."

    messages = [
        SystemMessage(system_prompt),
        HumanMessage(human_message)
    ]
    
    try:
        print("🤖 Sending request to the model...")

        respuesta = model.invoke(messages)

        if hasattr(respuesta, 'content'):
            contenido = respuesta.content
        else:
            contenido = str(respuesta)
        
        print("\n📋 MODEL RESPONSE:")
        print("-" * 50)
        print(contenido)
        print("-" * 50)
        
        lineas = contenido.split('\n')
        preguntas = []
        
        for linea in lineas:
            linea = linea.strip()
            if linea and any(linea.startswith(f"{i}.") for i in range(1, NUM_PREGUNTAS + 1)):
                pregunta = linea
                for i in range(1, NUM_PREGUNTAS + 1):
                    if pregunta.startswith(f"{i}."):
                        pregunta = pregunta[len(f"{i}."):].strip()
                        break
                if len(pregunta) > 10:
                    preguntas.append(pregunta)
        
        print(f"\n✅ PREGUNTAS EXTRAÍDAS ({len(preguntas)}):")
        print("=" * 60)
        
        for i, pregunta in enumerate(preguntas, 1):
            print(f"{i:2d}. {pregunta}")
        
        print(f"\n📊 SUMMARY:")
        print(f"- Questions generated: {len(preguntas)}")
        print(f"- Type: Técnicas generales de café")
        print(f"- Not based document")
        
        return preguntas
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

In [93]:
preguntas_generales = test_preguntas_cafe_sin_texto()

🤖 Enviando solicitud al modelo...

📋 RESPUESTA DEL MODELO:
--------------------------------------------------
1. ¿Cuál es la densidad óptima de siembra para Coffea arabica en sistemas de producción de alta productividad, considerando el manejo de sombra y topografía en zonas cafeteras colombianas?

2. ¿Cómo influye el manejo nutricional integral, con énfasis en relaciones C/N y aplicación foliar de micronutrientes, en el desarrollo vegetativo y la calidad de la taza en café?

3. ¿Cuáles son los principales signos visuales y métodos de diagnóstico temprano de la roya del café (Hemileia vastatrix) y cuál es el protocolo recomendado para su manejo integrado, incluyendo variedades resistentes y aplicaciones fitosanitarias?

4. ¿Qué criterios fisiológicos y productivos se emplean para seleccionar la entrada y la frecuencia de podas de renovación y formación en cafetales, y cómo impactan en la longevidad y productividad de la plantación?

5. ¿Cuáles son las características agronómicas, produ

In [94]:
preguntas_generales

['¿Cuál es la densidad óptima de siembra para Coffea arabica en sistemas de producción de alta productividad, considerando el manejo de sombra y topografía en zonas cafeteras colombianas?',
 '¿Cómo influye el manejo nutricional integral, con énfasis en relaciones C/N y aplicación foliar de micronutrientes, en el desarrollo vegetativo y la calidad de la taza en café?',
 '¿Cuáles son los principales signos visuales y métodos de diagnóstico temprano de la roya del café (Hemileia vastatrix) y cuál es el protocolo recomendado para su manejo integrado, incluyendo variedades resistentes y aplicaciones fitosanitarias?',
 '¿Qué criterios fisiológicos y productivos se emplean para seleccionar la entrada y la frecuencia de podas de renovación y formación en cafetales, y cómo impactan en la longevidad y productividad de la plantación?',
 '¿Cuáles son las características agronómicas, productivas y de calidad de taza diferenciadas entre las variedades Castillo, Cenicafé 1 y Geisha, y en qué condicio